# Préparation

In [69]:
%cd /content
!rm -rf test-zoomLDM
!git clone https://github.com/misterstab/test-zoomLDM.git
%cd test-zoomLDM

/content
Cloning into 'test-zoomLDM'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 186 (delta 80), reused 150 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (186/186), 23.08 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/test-zoomLDM


In [70]:
import sys
sys.path.append('/content/test-zoomLDM')
sys.path.insert(0, "../")

# Import des librairies nécessaires

In [68]:
!pip install -r requirements.txt

Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip (from -r requirements.txt (line 50))
  Cloning https://github.com/openai/CLIP.git (to revision main) to ./src/clip
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /content/test-zoomLDM/src/clip
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Obtaining file:///content/test-zoomLDM (from -r requirements.txt (line 51))
  Preparing metadata (setup.py) ... done
  Using cached black-23.7.0-py3-none-any.whl.metadata (63 kB)
  Using cached chardet-5.1.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached fairscale-0.4.13.tar.gz (266 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fire-0.7.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached invisibl

In [ ]:
from datasets import load_dataset
from omegaconf import OmegaConf

import numpy as np
import torch
from einops import rearrange
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download
from IPython.utils import io
from torch.utils.data import DataLoader

from ldm.util import instantiate_from_config
from utils import collate_fn

from PIL import Image

from large_image_gen.resizer import Resizer
from large_image_gen.utils import model_pred, decode_large_image

import torch.nn.functional as F
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [ ]:
model_dir = snapshot_download(repo_id="StonyBrook-CVLab/ZoomLDM")
print(f"Modèle téléchargé dans : {model_dir}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

naip/weights.ckpt:   0%|          | 0.00/3.58G [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

README.md:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

brca/weights.ckpt:   0%|          | 0.00/3.58G [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

Modèle téléchargé dans : /root/.cache/huggingface/hub/models--StonyBrook-CVLab--ZoomLDM/snapshots/433088972f0e96398ff9c54967a21fc3303eb6a5


In [ ]:
config = OmegaConf.load("configs/zoomldm_brca.yaml")
model = instantiate_from_config(config.model)
model.load_state_dict(torch.load('chemin/vers/le/pesage/model.ckpt'), strict=False)